**Load data as obtained from the Microplate reader and build a dataframe with the raw values**

Run this chunk to generate a dataframe (df) with raw values

In [ ]:
import pandas as pd

# Treatment labels (16 treatments × 4 replicates)
treatments = [
    "NC", "Dip8", "Dip16", "Dip32",
    "T-720", "T-360", "T-180",
    "Dip8/T-720", "Dip8/T-360", "Dip8/T-180",
    "Dip16/T-720", "Dip16/T-360", "Dip16/T-180",
    "Dip32/T-720", "Dip32/T-360", "Dip32/T-180"
]

# Load the 96 wells plate data
df_plate = pd.read_csv("dip_tuc_1.csv", header=None, skiprows=45, sep= ",")


# Drop first column (A-H) and replace "EP" with NA
df_values = df_plate.drop(columns=[0]).astype(str).replace(r'^\s*EP\s*$', pd.NA, regex=True)

# Initialize result container
data = []

# Current treatment index
treatment_index = 0

# Loop over columns
for col in range(df_values.shape[1]):
    col_values = df_values.iloc[:, col]

    # Extract all valid (non-na) values from the column
    valid_vals = col_values.dropna().tolist()

    # Process in groups of 4 (replicates)
    for i in range(0, len(valid_vals), 4):
        reps = valid_vals[i:i+4]
        if len(reps) == 4 and treatment_index < len(treatments):
            data.append([treatments[treatment_index]] + reps)
            treatment_index += 1

# Build DataFrame with raw data
df_raw = pd.DataFrame(data, columns=["treatment", "rep1", "rep2", "rep3", "rep4"])

#save raw data. If needed, uncomment the next line
#df_raw.to_csv("df_raw.csv", index=False)


**Build a dataframe adjusted by NC**

Run the previous chunk to generate the raw data frame and this chunk to generate a negative control adjusted dataframe. This df includes mean and standard deviation. The reads are expressed as viability (%).
You can use this df to generate a bar plot with error bars using the R script "Drug screenning".

In [ ]:
import numpy as np

# Rename for clarity and to keep both files
df_plot = df_raw.copy()

# Calculate mean of the negative control (NC)
nc_values = df_plot[df_plot["treatment"] == "NC"].iloc[0, 1:5].values.astype(float)
nc_mean = np.mean(nc_values)

# Normalize OD values to viability
for rep in ["rep1", "rep2", "rep3", "rep4"]:
    df_plot[rep] = df_plot[rep].astype(float) / nc_mean * 100

# Add 'mean' and 'SD' columns
df_plot["mean"] = df_plot[["rep1", "rep2", "rep3", "rep4"]].mean(axis=1)
df_plot["SD"] = df_plot[["rep1", "rep2", "rep3", "rep4"]].std(axis=1)

# Preview
print(df_plot)

# Save the normalized data to plot
df_plot.to_csv("df_plot.csv", index=False)


**Built DataFrame for SynergyFinder**

Run this chunk to generate a df to calculate and visualize Synergy Scores for Drug Combinations using the "Drug screening" script in R

In [ ]:
import pandas as pd

# Load the 96-well plate data (raw, from microplate reader)
df_plate = pd.read_csv("dip_tuc_1.csv", header=None, skiprows=45, sep=",")
# Drop first column (A-H) and replace "EP" with NA
df_values = df_plate.drop(columns=[0]).astype(str).replace(r'^\s*EP\s*$', pd.NA, regex=True)

# Mapping of fixed layout required by SynergyFinder
# Format: (drug1, drug2, conc1, conc2, conc_unit1, conc_unit2)
treatment_layout = [
    ("Dipyridamole", "Tucidinostat", 0, 0, "mM", "uM"),      # NC
    ("Dipyridamole", "Tucidinostat", 8, 0, "mM", "uM"),
    ("Dipyridamole", "Tucidinostat", 16, 0, "mM", "uM"),
    ("Dipyridamole", "Tucidinostat", 32, 0, "mM", "uM"),
    ("Dipyridamole", "Tucidinostat", 0, 720, "mM", "uM"),
    ("Dipyridamole", "Tucidinostat", 0, 360, "mM", "uM"),
    ("Dipyridamole", "Tucidinostat", 0, 180, "mM", "uM"),
    ("Dipyridamole", "Tucidinostat", 8, 720, "mM", "uM"),
    ("Dipyridamole", "Tucidinostat", 8, 360, "mM", "uM"),
    ("Dipyridamole", "Tucidinostat", 8, 180, "mM", "uM"),
    ("Dipyridamole", "Tucidinostat", 16, 720, "mM", "uM"),
    ("Dipyridamole", "Tucidinostat", 16, 360, "mM", "uM"),
    ("Dipyridamole", "Tucidinostat", 16, 180, "mM", "uM"),
    ("Dipyridamole", "Tucidinostat", 32, 720, "mM", "uM"),
    ("Dipyridamole", "Tucidinostat", 32, 360, "mM", "uM"),
    ("Dipyridamole", "Tucidinostat", 32, 180, "mM", "uM"),
]

# Build long-format dataframe
synergy_rows = []
treatment_index = 0
nc_values = []

# Extract NC values for normalization
for col in range(df_values.shape[1]):
    col_vals = df_values.iloc[:, col].dropna().tolist()
    if treatment_index == 0 and len(col_vals) >= 4:
        for val in col_vals[:4]:
            try:
                od = float(str(val).replace(",", "."))
                nc_values.append(od)
            except ValueError:
                continue
        break  # NC is only in the first treatment (first column)

# Calculate NC mean
nc_mean = sum(nc_values) / len(nc_values)

#process each treatment
for col in range(df_values.shape[1]):
    col_vals = df_values.iloc[:, col].dropna().tolist()
    # Expect 4 replicates per treatment, change this number if needed
    if len(col_vals) >= 4 and treatment_index < len(treatment_layout):
        drug1, drug2, conc1, conc2, unit1, unit2 = treatment_layout[treatment_index]
        for val in col_vals[:4]:
            try:
                od = float(str(val).replace(",", "."))
                norm_response = (od / nc_mean) * 100
                synergy_rows.append({
                    "block_id": "1", #An ID for each drug combination tested (ie. dip and tuc =1)
                    "drug1": drug1,
                    "drug2": drug2,
                    "conc1": conc1,
                    "conc2": conc2,
                    "response": norm_response,
                    "conc_unit1": unit1,
                    "conc_unit2": unit2
                })
            except ValueError:
                print(f"Warning: could not convert value '{val}'")
        treatment_index += 1

# Create final synergy DataFrame
df_synergy = pd.DataFrame(synergy_rows)

# Save to CSV
df_synergy.to_csv("synergyfinder_df.csv", index=False)

print(df_synergy)